In [0]:
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
census = pd.read_csv("../dataset/dataset.csv")

In [0]:
census.head()

In [0]:
census['income_bracket'].unique()

In [0]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [0]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform Train Test Split on census data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Create the Feature Columns for tf.estimator

In [0]:
census.columns

In [0]:
import tensorflow as tf

In [0]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [0]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [0]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create the Input Function

In [0]:
input_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

#### Create a LinearClassifier model with tf.estimator

In [0]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

In [0]:
model.train(input_fn=input_func,steps=5000)

### Evaluation

In [0]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [0]:
predictions = list(model.predict(input_fn=pred_fn))

In [0]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [0]:
final_preds[:10]

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(y_test,final_preds))